In [28]:
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Input, Flatten, Dropout, Activation,  Dense, Embedding
from keras.callbacks import ModelCheckpoint
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import os
import yaml
from functools import partial
from sklearn.preprocessing import LabelEncoder

# PATHS
base = ''
features_fp = os.path.join(base, 'training_X_power.npy')
labels_fp = os.path.join(base, 'sample_class.csv')
config_fp = os.path.join(base, 'config_power.yaml')

# DATA
features = np.load(features_fp)
labels = pd.read_csv(labels_fp, delimiter=",", index_col=0)

def find_on_effects(path):
    effects = yaml.load(open(path, 'r'))
    used = effects.get('DataAugmentation')
    active = used.get('active')
    sustain = used.get('sustain')
    r = used.get('effects')
    random_subeffects = [[k for k,v in r.get(a).items() if v.get('state') == 'random'] for a in active]
    return {a: random_subeffects[ix] for ix, a in enumerate(active)}, active, sustain

def one_hot_binarize(labels, predict_effects):
    
    abbrv = [x.split('.') for x in predict_effects]
    abbrv = [(x[0][:2] + '.' + x[-1][:2]).upper() for x in abbrv]
    binarize = labels[predict_effects].fillna(0)
    binarize = binarize.where(binarize == 0).replace(np.nan, 1)
    def labelize(row, abbrv):
        return ':__'.join(e + str(int(r)) for e,r in zip(abbrv, list(tuple(row))))
    labels = binarize.apply(lambda x: labelize(x, abbrv), axis = 1)
    encoder = LabelEncoder()
    one_hot = np_utils.to_categorical(encoder.fit_transform(labels))
    states = list(encoder.classes_)
    return one_hot, states

def format_data(config = config_fp, predicted_effects = ['overdrive.gain_db', 
                                                         'reverb.reverberance', 
                                                         'chorus.delays'] ):
    
    random_fx, active, sustain = find_on_effects(config_fp)
    print ('Active effects: {}'.format(active))
    print ('Persisting {} during augmentation'.format(sustain))
    print ('Parameters allowed randomization {}'.format(random_fx))
    # Ideally generate effects
    print ('Effects to predict {}'.format(predicted_effects))
    binarized_y, states = one_hot_binarize(labels = globals()['labels'], predict_effects = predict_effects)
    [*data] = train_test_split(globals()['features'],binarized_y, shuffle = True, test_size = 0.2, random_state = 44)
    shapes = list(map(lambda x: x.shape, data))
    return data, shapes, states


# MAKE DATA

In [35]:
data, shapes, states = format_data()

Active effects: ['overdrive', 'reverb', 'chorus', 'phaser']
Persisting ['groko'] during augmentation
Parameters allowed randomization {'overdrive': ['gain_db'], 'reverb': ['reverberance'], 'chorus': [], 'phaser': []}
Effects to predict ['overdrive.gain_db', 'reverb.reverberance', 'chorus.delays']


In [36]:
X_train, X_val, Y_train, Y_val = data

In [40]:
X_train[0].shape

(205, 1)

In [43]:
# NOT USING FUNCTIONAL ARCHITECTURE YET

# SAMPLE ARCHITECTURE
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=X_train[0].shape[1:]))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())

model2 = Sequential()
model2.add(model)
# USING BINARIZER CLASSES
model2.add(Dense(len(states), activation = "softmax"))

model1 = Sequential()
model1.add(model)
model1.add(Dense(1, activation = "relu"))

opt = opt = keras.optimizers.Adam(lr=0.00001)

model1.compile(optimizer= opt, loss=['mae'], metrics =["mae"])
model2.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])


ValueError: Input 0 is incompatible with layer conv1d_10: expected ndim=3, found ndim=2